In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/04/16 07:08:04 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 172.24.150.144 instead (on interface eth0)
23/04/16 07:08:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/16 07:08:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/16 07:08:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [6]:
df_green.show(10)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-27 23:23:13|  2020-01-27 23:33:22|                 N|         1|         260|         223|              1|         2.98|       11.5|  0.5|    0.

In [5]:
df_green.createOrReplaceTempView('green')

In [17]:
df_green_revenue = spark.sql("""

    select 
    date_trunc('hour', lpep_pickup_datetime) as hour,
    PULocationID,
    sum(total_amount) as amount,
    count(*) as num_trips
    from green
    where lpep_pickup_datetime>='2020'
    group by 1,2
    order by 1,2

""")

In [18]:
df_green_revenue.show()

+-------------------+------------+------------------+---------+
|               hour|PULocationID|            amount|num_trips|
+-------------------+------------+------------------+---------+
|2020-01-01 00:00:00|           7| 769.7299999999994|       45|
|2020-01-01 00:00:00|          17|            195.03|        9|
|2020-01-01 00:00:00|          18|               7.8|        1|
|2020-01-01 00:00:00|          22|              15.8|        1|
|2020-01-01 00:00:00|          24|              87.6|        3|
|2020-01-01 00:00:00|          25| 531.0000000000001|       26|
|2020-01-01 00:00:00|          29|              61.3|        1|
|2020-01-01 00:00:00|          32| 68.94999999999999|        2|
|2020-01-01 00:00:00|          33|317.27000000000004|       11|
|2020-01-01 00:00:00|          35|            129.96|        5|
|2020-01-01 00:00:00|          36|295.34000000000003|       11|
|2020-01-01 00:00:00|          37|            175.67|        6|
|2020-01-01 00:00:00|          38| 98.78

In [21]:
df_yellow.createOrReplaceTempView('yellow')

In [19]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [22]:
df_yellow_revenue = spark.sql("""

    select 
    date_trunc('hour', tpep_pickup_datetime) as hour,
    PULocationID,
    sum(total_amount) as amount,
    count(*) as num_trips
    from yellow
    where tpep_pickup_datetime>='2020'
    group by 1,2
    order by 1,2

""")

In [23]:
df_yellow_revenue.show()

+-------------------+------------+------------------+---------+
|               hour|PULocationID|            amount|num_trips|
+-------------------+------------+------------------+---------+
|2020-01-01 00:00:00|           3|              25.0|        1|
|2020-01-01 00:00:00|           4|1004.3000000000003|       57|
|2020-01-01 00:00:00|           7| 455.1700000000001|       38|
|2020-01-01 00:00:00|          10|             42.41|        2|
|2020-01-01 00:00:00|          12|106.99999999999999|        6|
|2020-01-01 00:00:00|          13|            1214.8|       56|
|2020-01-01 00:00:00|          14|               8.8|        1|
|2020-01-01 00:00:00|          15|             34.09|        1|
|2020-01-01 00:00:00|          17|220.20999999999998|        8|
|2020-01-01 00:00:00|          18|               5.8|        1|
|2020-01-01 00:00:00|          24| 754.9500000000003|       45|
|2020-01-01 00:00:00|          25|            324.35|       16|
|2020-01-01 00:00:00|          32|      

In [25]:
df_yellow_revenue.write.mode('overwrite').parquet('data/report/revenue/yellow/')

23/04/16 07:23:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [28]:
df_green_revenue.coalesce(5).write.mode('overwrite').parquet('data/report/revenue/green/')